In [25]:
import fitz  # PyMuPDF
from tqdm import tqdm  # For a progress bar

def extract_text_from_pdf(pdf_path, output_txt_path=None):
    # Open the PDF file
    doc = fitz.open(pdf_path)
    
    full_text = []
    
    print(f"Processing {len(doc)} pages...")
    
    # Iterate through each page
    for page_num in tqdm(range(len(doc))):
        page = doc.load_page(page_num)
        
        # Extract text. "text" mode gives plain text.
        # You can also use "blocks" to get text with coordinates if needed later.
        text = page.get_text("text").replace("""у
*К
б""", "")
        
        # Optional: Add a page marker to know where text came from
        # page_marker = f"\n\n--- Page {page_num + 1} ---\n\n"
        full_text.append(text)
        
    joined_text = "".join(full_text)
    
    # Save to file if path is provided
    if output_txt_path:
        with open(output_txt_path, "w", encoding="utf-8") as f:
            f.write(joined_text)
        print(f"Done! Text saved to {output_txt_path}")
        
    return full_text

# --- Usage ---
pdf_file = "data/Kazakh_history_6_sample.pdf"  # <--- Change this
output_file = "data/book_text_sample.txt"

# Run the function
text_content = extract_text_from_pdf(pdf_file, output_file)

# Check the first 500 characters
print(text_content[:500])

Processing 4 pages...


100%|██████████| 4/4 [00:00<00:00, 163.68it/s]

Done! Text saved to data/book_text_sample.txt
['7\nБIРIНШI \nБӨЛІМ\nТҮРКІ КЕЗЕҢIНДЕГI ҚАЗАҚСТАН ТАРИХЫ \n(VI–ХIII ғасырдың басы)\nБiрiншi тарау\nVІ–ІХ ҒАСЫРЛАРДАҒЫ ҚАЗАҚСТАН\n \n§1.  \nТҮРІК ҚАҒАНАТЫ (552–603 жылдар)\nТақырыпты оқу барысында:\n– Түрік қағанатының қалыптасу тарихын білесіңдер;\n– мемлекеттің құрылуы мен нығаюының себебін түсінесіңдер;\n– қағанат, ашина, ел, бұдун, бек, ер, этникалық сана-сезім ұғымдарымен \nтанысасыңдар. Оқиғалардың өзара байланысы арқылы Түрік қағанаты \nқұрылуының тарихи маңызын анықтайсыңдар.\nТүркілер қандай оқиғаға бай\xad\nланысты және Қытай жылна\xad\nмаларында алғаш қашан ата\xad\nлатындығын анықтаңдар.\nТүрік қағанатының құрылу \nбарысын түсіндіріңдер. Түрік \nқағанаты қашан және қай жер\xad\nде құрылды?\nТүрiк қағанатының құрылуы. Қытай деректерi бойынша түркілер \nалғашқы кезде Қытай шегарасының батысында, Алтай тауларының \nетегiнде өмiр сүрдi. Кейін түрiк тайпаларының көсемi Бумынның \nбасшылығымен иелiктерiн Хуанхэ жағалауларына дейін кеңе

In [6]:
file_path = "data/Kazakh_history_6_sample.pdf"

In [10]:
from pdf2image import convert_from_path
from pytesseract import image_to_string

def ocr_pdf_pages(pdf_path, dpi=300):
    """Конвертирует PDF в изображения и запускает pytesseract по каждой странице."""
    images = convert_from_path(pdf_path, dpi=dpi)
    texts = []
    for img in images:
        txt = image_to_string(img, lang="kz")
        texts.append(txt)
    return texts

pdf_images = images = convert_from_path(file_path, dpi=300)
pdf_images

[<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=2056x2634>,
 <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=2056x2634>,
 <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=2056x2634>,
 <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=2056x2634>]

In [14]:
import base64
from openai import OpenAI
import os
import io
from dotenv import load_dotenv

load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


# def encode_image(image_path):
#     with open(image_path, "rb") as image_file:
#         return base64.b64encode(image_file.read()).decode('utf-8')

# base64_image = encode_image("page_1.jpg")

img_base64 = []

for img in images:
    # 1. Создаем буфер в оперативной памяти
    buffered = io.BytesIO()
    
    # 2. Сохраняем PIL Image в буфер в формате JPEG (или PNG)
    img.save(buffered, format="JPEG")
    
    # 3. Получаем байты из буфера и кодируем в base64
    img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
    # 4. Добавляем результат в список
    img_base64.append(img_str)

img_base64

['...']

In [64]:
metrics = {}

In [ ]:
model_names = [
    "gpt-4o",
    "gpt-4o-mini",
    "gpt-4.1"
    "gpt-4.1-mini",
    "gpt-4.1-nano",
    "gpt-5",
    "gpt-5-mini",
    "gpt-5-nano"
]

In [128]:
import time
model_name = "gpt-5"
text_gpt_4o = []
time_start = time.time()

for img in img_base64:
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "Извлеки весь текст из этого изображения. Сохраняй структуру и форматирование. Не добавляй лишних комментариев. верни markdown файл"},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{img}"}}
                ],
            }
        ],
        temperature=1, # Твой контроль!
    )
    text_gpt_4o.append(response.choices[0].message.content)

model_output = []
for text in text_gpt_4o:
    model_output.append(text.replace("```markdown","").replace("```", "") + "\n")

time_spent = time.time() - time_start
print(f"{time_spent} seconds")

model_output

576.2325720787048 seconds


['# Все учебники Казахстана на OKULYK.KZ\n\n## БІРІНШІ БӨЛІМ\n### ТҮРІК КЕЗЕҢІНДЕГІ ҚАЗАҚСТАН ТАРИХЫ (VI–XIII ғасырдың басы)\n\n#### Бірінші тарау  \nVI–IX ҒАСЫРЛАРДАҒЫ ҚАЗАҚСТАН\n\n§1. ТҮРІК ҚАҒАНАТЫ (552–603 жылдар)\n\nТақырыпты оқу барысында:\n- Түрік қағанатының қалыптасу тарихын білесіңдер;\n- мемлекеттің құрылуы мен нығаюының себебін түсінесіңдер;\n- қағанат, ашина, ел, бүдун, бек, ер, этникалық сана-сезім ұғымдарымен танысасыңдар;\n- Оқиғалардың өзара байланысы арқылы Түрік қағанаты құрылуының тарихи маңызын анықтайсыздар.\n\nТүрік қағанатының құрылуы. Қытай деректері бойынша түркілер алғашқы кезде Қытай шекарасының батысында, Алтай тауларының етегінде өмір сүрді. Кейін түрік тайпаларының көсемі Бумынның басшылығымен иеліктерін Хуанхэ жағалауларына дейін кеңейтті. Батыс Қытай билеушісі жақын қоныстанған түркілер жайлы дерек жинай бастайды. Бұл деректер түрік тарихының ерте кезеңі туралы негізгі білім көзіне айналды. Қытай деректерінде түркілерге қатысты алғашқы мәліметтер 542 жы

In [129]:


page1 = """7 БIРIНШI 
БӨЛІМ
ТҮРКІ КЕЗЕҢIНДЕГI ҚАЗАҚСТАН ТАРИХЫ 
(VI–ХIII ғасырдың басы)
Бiрiншi тарау
VІ–ІХ ҒАСЫРЛАРДАҒЫ ҚАЗАҚСТАН
 
§1.  
ТҮРІК ҚАҒАНАТЫ (552–603 жылдар)
Тақырыпты оқу барысында:
– Түрік қағанатының қалыптасу тарихын білесіңдер;
– мемлекеттің құрылуы мен нығаюының себебін түсінесіңдер;
– қағанат, ашина, ел, бұдун, бек, ер, этникалық сана-сезім ұғымдарымен 
танысасыңдар. Оқиғалардың өзара байланысы арқылы Түрік қағанаты 
құрылуының тарихи маңызын анықтайсыңдар.
Түркілер қандай оқиғаға бай­ланысты және Қытай жылна­маларында алғаш қашан ата­латындығын анықтаңдар.
Түрік қағанатының құрылу 
барысын түсіндіріңдер. Түрік 
қағанаты қашан және қай жер­де құрылды?
Түрiк қағанатының құрылуы. Қытай деректерi бойынша түркілер 
алғашқы кезде Қытай шегарасының батысында, Алтай тауларының 
етегiнде өмiр сүрдi. Кейін түрiк тайпаларының көсемi Бумынның 
басшылығымен иелiктерiн Хуанхэ жағалауларына дейін кеңейтті. Батыс Қытай билеушiсi жақын қоныстанған түркiлер жайлы дерек жинай бастайды. Бұл деректер түрiк тарихының ерте кезеңi туралы негiзгi 
білім көзiне айналды. Қытай деректерiнде түркілерге қатысты алғашқы 
мәлiметтер 542 жылдан бастап кездеседі. Түркілер жужан қағанына 
салықты темір түрінде төлеген.
«Түрік» сөзi «берік, күштi» деген мағынаны бiлдiріп, билiк етушi 
ашина әулетiне байланыс ты айтылды. Кейiн олардың қарамағындағы 
барлық тайпалардың ортақ атауына айналды. 
Бұл тайпалардың біраз бөлігі теле деп аталды. 
Жужандардың тепкісін көрген түркілер 
оларға қарсы бас көтере бастайды. Бумын қалың қолмен жужандарға шабуыл жасайды. 
552 жылы жужандар әс керi талқандалады. 
Бумын түркі елiнiң қағаны болып жарияланады. Орталық Азиядағы жаңа мемлекет – 
Түрiк қағанаты осылай пайда болды."""

page2 = """8
Түрiк қағанатының негiзiн қалаушы Бумын қайтыс болғаннан кейін билiкке келген 
Мұқан қаған (553–572 жылдар) жужандарды 
талқандауды аяқтайды.
Түрік қағанатының нығаюы. Мұқан қаған 
түркілердiң Орталық Азия мен Оңтүстiк Сiбiрдегi үстемдiгiн бекiттi. Иштеми (Iстеми) қаған 
қазiргi Қазақстан, Орталық Азия аумағын бағындырып, Едiл мен Солтүстiк Кавказға шықты.
Қытай жылнамаларындағы деректер бойынша Мұқан қаған «шегара (Ұлы қорған) сыртындағы барлық иелiктердiң зәресiн ұшырды». 
VI ғасырдың 60-жылдары Түрiк қағанаты сол кездегi iрi мемлекеттер – Византия, Иран, Қытаймен өзара қарым-қатынас жасады. Түрiк 
қағанаты нығайған кезiнде (VI ғасырдың 70-жылдары) Маньчжуриядан 
Босфорға дейiнгi жердi алып жатты. Иштеми қаған тұсында түркiлер 
әскери жағынан қуатты империяға айналды.
Түркілердiң батысқа жылжуы тек 
жаулап алушылық қана емес, түркi 
тайпаларының iрi көшi-қоны на ұласты. Жергiлiктi тайпалар түркілерден 
құралған мемлекетке қосылды немесе 
Шығыс Еуропаға қарай жылжыды.
VI ғасырдың 80-жылдарының соңында түркілер Парсы елімен 
 одақ тасты. Оңтүстік Қазақстан мен Орталық Азиядағы қуатты эфталиттер 
мемлекетін талқандады. Эфталиттер мұрасын бөлуге байланысты одақтастар арасында жанжал шығып, ол тез арада шешілді. Түркілердің өз 
әскерiн керi алып кеткенi үшiн Парсы жағы оларға үлкен көлемде алым 
төлеуге міндеттенді.
Түркілер Орталық Азияны жаулап алып, Қытайдан Жерорта теңiзi 
елдерiне баратын Ұлы Жiбек жолының едәуір бөлігіне ие болды. Жібек 
саудасы түркі қағандарына орасан зор кіріс әкелді. Жiбек матасын сатып 
алушы негiзiнен Византия еді. 
568 жылы түркі елшісі Маниах Византияға қағанаттың елшiлiгiн 
басқарып барды. Император сарайы түркі елшiлiгiн аса жоғары құрметпен 
қабылдады. Түркілер мен Византия арасында парсыларға қарсы әскери-Ұлттық сана-сезім – ха­лықтың өзіндік ерекшелік­тері мен әлемдегі өз 
ор­нын ұғынуы, мәдени­тіл ор тақтығы.
Түркілер қашан және неліктен 
жужандарға қарсы көтерілді? 
(Ұлттық сана­сезім ұғымын 
пайдаланыңдар).
Жужан мемлекеті V ғасырда Монғолия 
мен Батыс Маньчжурия аумағында 
көшпелі тайпалар одағынан құрылған. 
Түркілер талқандаған жужандардың 
бір бөлігі Корея мен Солтүстік Қытайға, 
басқалары – батысқа қоныс аударды."""

page3 = """10

Ежелгi түркі жауынгерлерi

Қаған мемлекетті басқарды, жоғарғы сот мiндетiн атқарды, әскерге басшылық етті. Мемлекетті басқаруда тайпаның ақсүйек қауымына 
сүйендi. Әскери және азаматтық билікті басқаратын жүйе – ябғу, шад 
т.б. құрылды.
Билеуші әулеттің жасы жағынан ең үлкені қаған сайланды. Мұның 
өзi кәмелетке жетпеген ханзаданы таққа отырғызудан сақтап, елдiң 
қауiпсiздігін қамтамасыз етті. Билiк үнемi тәжiрибелi адамдардың қолында болды. Ұлы қаған ордасы түркілер дiң 
байырғы қонысы – Алтайда орналасты.
Түрiк мемлекетiнде қарапайым халық 
бұдун (мемлекеттiң қатардағы құрамы) 
аталды. «Бектер» ақсүйектер билiк тобын құрады.
Қаған ашина руынан шыққан түркi елiндегi ақсүйек қауымының ең таңдаулы өкілі 
саналды. Зерттеушiлер «ашина» терминiн 
әрқилы саралайды, бiреулерi бұл сөздi «текті 
қасқыр», ал басқалары «аспаннан жара тылған» деп түсiндiредi.
Түркілердiң шыққан тегі бір болған дықтан, олардың жерге және соғыс тан түс кен """

page4 = """12
2.2. 545 жылы түркілер тарих сахнасына шыққан кезде оларға елшілігі келген 
мемлекет
 
А) Византия
 
Ә) Қытай
 
Б) Русь
 
В) Парсы
 
Г) Аштархан билеушiлерi
2.3. Бумын бастаған күрестегі оның басты жауы
 
А) ғұндар
 
Ә) үйсiндер
 
Б) жужандар
 
В) монғолдар
 
Г) меркiттер 
2.4. Орталық Азиядағы жаңа мемлекет – Түрiк қағанаты пайда болған жыл
 
А) 572 жыл
 
Ә) 542 жыл
 
Б) 532 жыл
 
В) 562 жыл
 
Г) 552 жыл
Бiртұтас Түрiк қағанатының құлдырауы негізінде жаңа түркі мемлекет терi қалыптасты. Осы мемлекеттер шеңберiнде Ұлы Далада көне түркi 
халықтары одан әрі дамып, айналасындағы тайпаларға ықпал ете бастады. 
«Ұлы Дала» ежелден қалыптасқан ұғым. Ұлы Далаға Алтайдан Қара теңізге 
дейінгі кең-байтақ аумақ жатады. Оның негізін қазақтың жері құрайды. 
Ұлы Даланы мекендеген түркілер өз кезінде қуатты мемлекеттерін құрған. 
  §2. 
БАТЫС ТҮРIК (603–704 жылдар) ЖӘНЕ ШЫҒЫС  
 
 
ТҮРIК (682–744 жылдар) ҚАҒАНАТТАРЫ
Бүгінгі сабақта мына сұрақтарға жауап бере аласыңдар:
– Батыс түркілері неліктен өз мемлекетін құрды?
– Батыс және Шығыс Түрік қағанаттарының даму ерекшеліктері мен ұқсас­
тықтары және айырмашылықтары неде?
– ябғу, шад, елтебер, тархан, бұйрық, қара бұдун, тат ұғымдары нені 
білдіреді? 
Тақырыпты оқу барысында:
– ерте түрік қағанаттарының сыртқы саясатының негізгі бағыттарын анық­
тайсыңдар;
–  «Ұлы Дала» ұғымымен танысасыңдар.

"""



In [130]:
import jiwer

def calculate_cer_wer(ground_truth, model_output):
    # Для CER (Character Error Rate)
    transformation_cer = jiwer.Compose([
        jiwer.ToLowerCase(),
        jiwer.RemovePunctuation(),
        jiwer.RemoveMultipleSpaces(),
        jiwer.Strip(),
        jiwer.ReduceToListOfListOfChars() # <--- ОБЯЗАТЕЛЬНО для CER
    ])

    cer_value = jiwer.cer(
        ground_truth, 
        model_output, 
        reference_transform=transformation_cer, 
        hypothesis_transform=transformation_cer
    )
    transformation_wer = jiwer.Compose([
    jiwer.ToLowerCase(),
    jiwer.RemovePunctuation(),
    jiwer.RemoveMultipleSpaces(),
    jiwer.Strip(),
    jiwer.ReduceToListOfListOfWords() # <--- ОБЯЗАТЕЛЬНО для WER
    ])

    wer_value = jiwer.wer(
        ground_truth, 
        model_output, 
        reference_transform=transformation_wer, 
        hypothesis_transform=transformation_wer
    )
    
    print(f"CER: {cer_value:.2%}")
    print(f"WER: {wer_value:.2%}")  

In [131]:
ground_truth = [page1, page2, page3, page4]

calculate_cer_wer(ground_truth, model_output)

CER: 45.52%
WER: 70.75%


In [132]:
import re

def get_bag_of_words_metrics(ground_truth_text, ocr_text):
    # 1. Preprocessing function
    def tokenize(text):
        # Lowercase and remove non-alphanumeric chars (keep simple)
        text = text.lower()
        # Split by whitespace and remove punctuation
        words = re.findall(r'\w+', text)
        print(words)
        return set(words) # Convert to a SET to ignore order and duplicates

    # 2. Create Sets
    gt_set = tokenize(ground_truth_text)
    ocr_set = tokenize(ocr_text)

    # 3. Calculate Intersections
    # True Positives: Words in both
    tp = len(gt_set.intersection(ocr_set))
    
    # False Positives: Words in OCR but not in GT (Hallucinations)
    fp = len(ocr_set - gt_set)
    
    # False Negatives: Words in GT but missed by OCR
    fn = len(gt_set - ocr_set)

    # 4. Calculate Metrics
    # Handle division by zero carefully
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    
    # Jaccard Index (Intersection over Union)
    union = len(gt_set.union(ocr_set))
    jaccard = tp / union if union > 0 else 0.0

    return {
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "jaccard": jaccard,
        "missed_words": list(gt_set - ocr_set) # Useful for debugging!
    }

# --- Example Usage ---

# Ground Truth (Order doesn't matter)
ground_truth_text = "".join(ground_truth)
model_output_text = "".join(model_output)

metrics_per_model = get_bag_of_words_metrics(ground_truth_text, model_output_text)
metrics_per_model["time_spent"] = time_spent

print(f"Recall:    {metrics_per_model['recall']:.2%}")   # How many keywords did we catch?
print(f"Precision: {metrics_per_model['precision']:.2%}")# How much of the OCR output was valid?
print(f"F1 Score:  {metrics_per_model['f1_score']:.2%}") # The overall quality score
print(f"Missed:    {metrics_per_model['missed_words']}") # Shows ['vendor', 'acme']

metrics[model_name] = metrics_per_model

['7', 'бiрiншi', 'бөлім', 'түркі', 'кезеңiндегi', 'қазақстан', 'тарихы', 'vi', 'хiii', 'ғасырдың', 'басы', 'бiрiншi', 'тарау', 'vі', 'іх', 'ғасырлардағы', 'қазақстан', '1', 'түрік', 'қағанаты', '552', '603', 'жылдар', 'тақырыпты', 'оқу', 'барысында', 'түрік', 'қағанатының', 'қалыптасу', 'тарихын', 'білесіңдер', 'мемлекеттің', 'құрылуы', 'мен', 'нығаюының', 'себебін', 'түсінесіңдер', 'қағанат', 'ашина', 'ел', 'бұдун', 'бек', 'ер', 'этникалық', 'сана', 'сезім', 'ұғымдарымен', 'танысасыңдар', 'оқиғалардың', 'өзара', 'байланысы', 'арқылы', 'түрік', 'қағанаты', 'құрылуының', 'тарихи', 'маңызын', 'анықтайсыңдар', 'түркілер', 'қандай', 'оқиғаға', 'бай', 'ланысты', 'және', 'қытай', 'жылна', 'маларында', 'алғаш', 'қашан', 'ата', 'латындығын', 'анықтаңдар', 'түрік', 'қағанатының', 'құрылу', 'барысын', 'түсіндіріңдер', 'түрік', 'қағанаты', 'қашан', 'және', 'қай', 'жер', 'де', 'құрылды', 'түрiк', 'қағанатының', 'құрылуы', 'қытай', 'деректерi', 'бойынша', 'түркілер', 'алғашқы', 'кезде', 'қытай', 'ш

In [133]:
for key,val in metrics.items():
    print(f"{key}: {val["f1_score"]:.2%}, Time: {val.get("time_spent")} sec")

gpt-4o: 73.98%, Time: None sec
gpt-4o-mini: 59.02%, Time: None sec
gpt-4.1-mini: 79.61%, Time: None sec
gpt-4.1-nano: 72.76%, Time: None sec
gpt-4.1: 75.62%, Time: None sec
gpt-5-mini: 76.52%, Time: 124.84835886955261 sec
gpt-5-nano: 42.63%, Time: 154.12854313850403 sec
gpt-5: 74.71%, Time: 576.2325720787048 sec


In [134]:
with open("data/book_text_extracted.txt", "r", encoding="utf-8") as f:
    text = f.read()
    print(len(text))

397901
